| **ATTRIBUTES**            |**MEANING**               |
|:----------------------|:-------------------------------------------------------------|
|**`Title`**            | Title of the manga (written in English phonetic)                                                |
|**`Score`**            | Score on the MyAnimeList site (MAL)                                                             |
|**`Vote`**             | Number of readers voting for the manga                                                          |
|**`Ranked`**           | Ranking of manga on the web MyAnimeList (MAL)                                                   |
|**`Popularity`**       | The popularity of the manga                                                                     |
|**`Members`**          | Number of readers who have this manga in their list                                             |
|**`Favorite`**         | Number of readers who love this manga                                                           |
|**`Type`**		        | Type (manga/manhwa/lightnovel...)                                                               |
|**`Volumes`**          | Number of volumes of manga                                                                      |
|**`Chapters`**         | Number of chapters of manga                                                                     |
|**`Status`**           | Status of the manga (ongoing, completed, on hiatus,...)                                         |
|**`Published`**        | Release time to the end time of the manga                                                       |
|**`Genres`**           | Genres of manga                                                                                 |
|**`Themes`**           | The themes of the manga                                                                         |
|**`Demographics`** 	| Target demographic (e.g., Shounen).                                                             |
|**`Serialization`** 	| Manga serialization information (e.g., Shounen Jump).                                           |
|**`Author`**           | Author of manga                                                                                 |
|**`Total Review`**     | Number of readers leaving comments on the manga                                                 |
|**`Type Review`**      | Number of readers for each comment category (Recommended / Mixed feeling / Not recommended)     |

In [1]:
!pip install requests-html

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: websockets
    Found existing installation: websockets 12.0
    Uninstalling websockets-12.0:
      Successfully uninstalled websockets-12.0


In [2]:
!pip install lxml_html_clean

In [3]:
import requests
from requests_html import HTMLSession
from bs4 import BeautifulSoup
import re
import nest_asyncio
import pandas as pd 
import datetime
import time

### Crawl data URLS

In [4]:
nest_asyncio.apply() 
session = HTMLSession()

In [5]:
listUrl1 = []

for i in range(0,253,50):
    # Url of the website to scrap
    url = f'https://myanimelist.net/topmanga.php?type=novels&limit={i}'

    # Get the html content
    html = requests.get(url).text

    # Parse the html content
    soup = BeautifulSoup(html, "html.parser")

    # Get the list of manga
    listItem = soup.find_all("td", {"class": "title al va-t clearfix word-break"})

    # Get the url of each manga
    for item in listItem:
        listUrl1.append(item.find('a').get('href'))

    # Print the number of manga urls collected
    print(f'{len(listUrl1)} urls collected', end='\r', flush=True)

In [6]:
listUrl = listUrl1
print(f'Total: {len(listUrl)} urls collected')

Total: 253 urls collected


In [7]:
with open("/kaggle/working/top_novel_collecting.txt", "w") as file:
    file.writelines(item + "\n" for item in listUrl)

### Crawl HTML content from the manga/light novel/... URLs

In [8]:
listHtml1 = []

for url in listUrl[0:253]:
    res = session.get(url)
    while len(res.text) < 4000:
        # Sleep for 10 minutes
        time.sleep(200)
        res = session.get(url)
        
    listHtml1.append(res.text)

    # Print the number of manga html collected
    print(f'{len(listHtml1)}/{len(listUrl)} manga html collected', end='\r', flush=True)

In [9]:
# Extract time of data collection to report for the project
now = datetime.datetime.now()
now = now.strftime("%Y-%m-%d")
print("Time of data collection: ", now)

Time of data collection:  2024-11-19


In [10]:
listHtml = listHtml1
print(f'Total: {len(listHtml)} manga html collected')

Total: 253 manga html collected


In [11]:
def extract_info(htmlComic):
    soup = BeautifulSoup(htmlComic, "html.parser")

    title = soup.find('span', {'itemprop': 'name'})
    if title is None:
        return None
    else:
        title_text = title.text.strip()
        title_english_span = title.find('span', {'class': 'title-english'})

        if title_english_span is not None:
            title_english_text = title_english_span.text.strip()
            title_text = title_text.replace(title_english_text, '')
            title = f'{title_text} ({title_english_text})'
        else:
            title = title_text
    # ratingValue = soup.find('span', {'itemprop': 'ratingValue'}).text
    # ratingCount = soup.find('span', {'itemprop': 'ratingCount'}).text
    # ranked = re.findall(r'\d+', soup.find('span', {'class': 'numbers ranked'}).text)[0]
    ratingValue_tag = soup.find('span', {'itemprop': 'ratingValue'})
    ratingValue = ratingValue_tag.text.strip() if ratingValue_tag else None
    
    ratingCount_tag = soup.find('span', {'itemprop': 'ratingCount'})
    ratingCount = ratingCount_tag.text.strip() if ratingCount_tag else None
    
    # ranked_tag = soup.find('span', {'class': 'numbers ranked'})
    # ranked = re.findall(r'\d+', ranked_tag.text)[0] if ranked_tag else None
    ranked_tag = soup.find('span', {'class': 'numbers ranked'})
    if ranked_tag:
        ranked_numbers = re.findall(r'\d+', ranked_tag.text)
        ranked = ranked_numbers[0] if ranked_numbers else None
    else:
        ranked = None
    popularity = re.findall(r'\d+', soup.find('span', {'class': 'numbers popularity'}).text)[0]

    volumes, chapters, status, published = '', '', '', ''
    genres, themes, authors, favorites, members = [], [], '', '', ''
    type_, demographic, serialization = '', '', ''

    for space in soup.find_all("div", {'class': 'spaceit_pad'}):
        text = space.text.strip()
        
        if 'Type:' in text:
            type_ = text.split(':', 1)[1].strip()
        elif 'Volumes:' in text:
            volumes = text.split(':', 1)[1].strip()
        elif 'Chapters:' in text:
            chapters = text.split(':', 1)[1].strip()
        elif 'Status:' in text:
            # Lấy nội dung sau thẻ <span class="dark_text">
            status = space.find('span', {'class': 'dark_text'}).next_sibling.strip()
        elif 'Published:' in text:
            published = text.split(':', 1)[1].strip()
        elif 'Genres:' in text or 'Genre:' in text:
            genres = [gen.text.strip() for gen in space.find_all('a')]
        elif 'Themes:' in text or 'Theme:' in text:
            # Lấy cả giá trị từ <a> và <span itemprop="genre">
            themes = [theme.text.strip() for theme in space.find_all('a')]
        elif 'Demographic:' in text or 'Demographics:' in text:
            demographic = space.find('a').text.strip()
        elif 'Serialization:' in text or 'Serializations:' in text:
            # serialization = space.find('a').text.strip()
            serialization_tag = space.find('a')  # Tìm thẻ <a>
            serialization = serialization_tag.text.strip() if serialization_tag else ''  # Kiểm tra nếu không tìm thấy
        elif 'Authors:' in text or 'Author:' in text:
            authors = text.split(':')[1].strip()
            # authors = space.find('a').text.strip()
            # author_tag = space.find('a')  # Tìm thẻ <a>
            # authors = author_tag.text.strip() if author_tag else ''  # Kiểm tra nếu không tìm thấy
        elif 'Favorites:' in text:
            favorites = text.split(':', 1)[1].strip()
        elif 'Members:' in text:
            members = text.split(':', 1)[1].strip()

    infoReviews = soup.find('div', {'class': 'manga-info-review__header mal-navbar'})
    totalReviews = re.findall(r'\d+', infoReviews.find('div', {'class': 'right'}).text)[0]

    typeReview = [
        int(re.findall(r'\d+', infoReviews.find('div', {'class': 'recommended'}).text)[0]),
        int(re.findall(r'\d+', infoReviews.find('div', {'class': 'mixed-feelings'}).text)[0]),
        int(re.findall(r'\d+', infoReviews.find('div', {'class': 'not-recommended'}).text)[0])
    ]

    return {
        "Title": title, "Score": ratingValue, "Vote": ratingCount,
        "Ranked": ranked, "Popularity": popularity, "Members": members,
        "Favorite": favorites, "Types": type_, "Volumes": volumes, 
        "Chapters": chapters, "Status": status, "Published": published, 
        "Genres": genres, "Themes": themes, "Demographic": demographic, "Serialization": serialization, 
        "Author": authors, "Total Review": totalReviews, "Type Review": typeReview
    }

# data_list = [extract_info(htmlComic) for htmlComic in listHtml if extract_info(htmlComic) is not None]
# df = pd.DataFrame(data_list)
data_list = []
for idx, htmlComic in enumerate(listHtml, start=1):
    result = extract_info(htmlComic)
    if result is not None:
        data_list.append(result)
    # In trạng thái sau khi duyệt mỗi phần tử
    print(f"Đã xử lý {idx}/{len(listHtml)} phần tử.", end='\r', flush=True)
    # print(f'{len(listUrl1)} urls collected', end='\r', flush=True)

df = pd.DataFrame(data_list)

In [12]:
df.head()

,Title,Score,Vote,Ranked,Popularity,Members,Favorite,Types,Volumes,Chapters,Status,Published,Genres,Themes,Demographic,Serialization,Author,Total Review,Type Review
0,Tian Guan Cifu (Heaven Official's Blessing: Ti...,9.05,6022,8,1579,"13,253","1,305",Novel,6,Unknown,Finished,"Feb 4, 2021 to Mar 31, 2022","[Action, Adventure, Boys Love, Supernatural]",[Historical],,,Mo Xiang Tong Xiu (Story),5,"[3, 1, 1]"
1,Omniscient Reader's Viewpoint,9.02,7701,11,1126,"17,527","1,653",Novel,20,105,Finished,"Jan 20, 2022 to Sep 11, 2023","[Action, Adventure, Fantasy]",[Urban Fantasy],,,sing N song (Story),20,"[19, 1, 0]"
2,Guimi Zhi Zhu (Lord of Mysteries),8.97,2222,15,3772,"5,764",487,Novel,18,366,Finished,"May 1, 2020 to Aug 1, 2023","[Action, Mystery, Supernatural, Suspense]",[],,,Ai Qianshui de Wuzei (Story),15,"[14, 0, 1]"
3,Mo Dao Zu Shi (Grandmaster of Demonic Cultivat...,8.91,4202,20,2549,"8,448",714,Novel,4,27,Finished,"Dec 8, 2016","[Action, Adventure, Boys Love, Mystery, Supern...",[Historical],,,Mo Xiang Tong Xiu (Story),3,"[1, 1, 1]"
4,Mikkakan no Koufuku (Three Days of Happiness),8.88,10871,25,557,"33,121","1,402",Novel,1,15,Finished,"Dec 25, 2013",[Drama],[],,,"Miaki, Sugaru (Story)",19,"[16, 1, 2]"


In [13]:
df.to_csv('/kaggle/working/raw_top_novels.csv', encoding='utf-8-sig', index=False)